### **Rechnung + Waschlohn (Lieferscheine) für interne Kunden als erledigt markieren. Diese werden nicht an SAP übertragen.**

In [ ]:
USE Salesianer;
GO

-- leere Lieferscheine bzgl. SAP-Übergabe erledigen
UPDATE LsKo SET InternKalkFix = 1, SentToSAP = -1
WHERE LsKo.Status >= 'Q'
  AND NOT EXISTS (SELECT LsPo.ID FROM LsPo WHERE LsPo.LsKoID = LsKo.ID AND  LsPo.Menge <> 0)
  AND (LsKo.SentToSAP = 0 OR LsKo.InternKalkFix = 0)
  AND LsKo.Datum < CAST(GETDATE() AS date);

GO

-- Lieferscheine/Waschlöhne von internen Kunden bzgl. SAP-Übergabe erledigen
UPDATE LsKo SET InternKalkFix = 1, SentToSAP = -1
FROM vsa, kunden
WHERE lsko.VsaID = vsa.id
  AND vsa.kundenid = kunden.id
  AND LsKo.Status >= 'Q'
  AND (LsKo.SentToSAP = 0 OR LsKo.InternKalkFix = 0)
  AND kunden.KdGfID NOT IN (SELECT ID FROM KdGf WHERE KurzBez IN (N'MED', N'GAST', N'JOB', N'SAEU', N'BM', N'RT', N'MIC'));

GO

-- Rechnungen von internen Kunden bzgl. SAP-Übergabe erledigen
UPDATE RechKo SET FiBuExpID = -2
FROM Kunden
WHERE RechKo.kundenID = Kunden.ID
  AND RechKo.FiBuExpID = -1
  AND RechKo.Status >= 'F'
  AND kunden.KdGfID NOT IN (SELECT ID FROM KdGf WHERE KurzBez IN (N'MED', N'GAST', N'JOB', N'SAEU', N'BM', N'RT', N'MIC'));

GO

### **Lieferscheine: fehlende Standorte nach-ermitteln und eintragen**

In [ ]:
USE Salesianer;
GO

DROP TABLE IF EXISTS #LsStandort;
GO

SELECT LsKo.ID LsKoID, LsKo.FahrtID, LsPo.ID LsPoID, LsKo.VsaID, LsPo.ProduktionID, Fahrt.ExpeditionID, KdBer.BereichID, Fahrt.ExpeditionID newExpeditionID,  LsPo.ProduktionID newProduktionID
INTO #LsStandort
FROM lsko, LsPo, Fahrt, KdArti, KDBer, Vsa
WHERE Lsko.senttosap = 0
  AND LsKo.Status >= 'Q'
  AND LsPo.LskoID = LsKo.ID
  AND LsKo.FahrtID = Fahrt.ID
  AND LsPo.KdArtiID = KdArti.ID
  AND KdArti.KdBerID = KdBer.ID
  AND LsKo.VsaID = Vsa.ID
  AND (Fahrt.ExpeditionID = -1 OR LsPo.ProduktionID = -1);

UPDATE #LsStandort SET newExpeditionID = StandBer.ExpeditionID
FROM StandBer, Vsa
WHERE Vsa.ID = #LsStandort.VsaID
  AND StandBer.StandKonID = Vsa.StandKonID
  AND StandBer.BereichID = #LsStandort.BereichID
  AND #LsStandort.ExpeditionID = -1;

UPDATE #LsStandort SET newProduktionID = StandBer.ProduktionID
FROM StandBer, Vsa
WHERE Vsa.ID = #LsStandort.VsaID
  AND StandBer.StandKonID = Vsa.StandKonID
  AND StandBer.BereichID = #LsStandort.BereichID
  AND #LsStandort.ExpeditionID = -1;

UPDATE #LsStandort SET newProduktionID = Kunden.StandortID
FROM Kunden, VSA
WHERE #LsStandort.VsaID = Vsa.ID
  AND Vsa.KundenID = Kunden.ID
  AND #LsStandort.newProduktionID = -1;
 
UPDATE #LsStandort SET newExpeditionID = Kunden.StandortID
FROM Kunden, VSA
WHERE #LsStandort.VsaID = Vsa.ID
  AND Vsa.KundenID = Kunden.ID
  AND #LsStandort.newExpeditionID = -1;

UPDATE LsPo SET ProduktionID = newProduktionID
FROM #LsStandort
WHERE #LsStandort.LsPoID = LsPo.ID
  AND LsPo.ProduktionID <> newProduktionID;
 
UPDATE Fahrt SET ExpeditionID = newExpeditionID
FROM #LsStandort
WHERE #LsStandort.FahrtID = Fahrt.ID
  AND Fahrt.ID > 0
  AND Fahrt.ExpeditionID <> newExpeditionID;
 
INSERT INTO Fahrt (PlanDatum, UrDatum, TourenID, ExpeditionID, AnlageUserID_)
SELECT DISTINCT LsKo.Datum, LsKo.Datum, LsKo.TourenID, newExpeditionID, 8888
FROM #LsStandort, LsKo
WHERE LsKo.ID = #LsStandort.LskoID
  AND #LsStandort.FahrtID = -1;

UPDATE LsKo SET FahrtID = Fahrt.ID
FROM #LsStandort, Fahrt
WHERE LsKo.ID = #LsStandort.LskoID
  AND LsKo.FahrtID = -1
  AND Fahrt.PlanDatum = LsKo.Datum
  AND Fahrt.TourenID = LsKo.TourenID
  AND Fahrt.AnlageUserID_ = 8888;

GO

### **Waschlohn ausrechnen**

Modul-Aufruf im AdvanTex: INKALKAPPLY;20210101;20210430  
Datumswerte entsprechend anpassen!

### **Waschlohn fixieren**

Modul-Aufruf im AdvanTex: INKALKFIX;20210101;20210430  
Datumswerte entsprechend anpassen!

### **Lieferscheine für Waschlohn-Übertragung in einen Zwischentabelle schreiben**

In [ ]:
USE Salesianer;
GO

IF OBJECT_ID(N'__LsInKalk') IS NOT NULL
  TRUNCATE TABLE __LsInKalk;
ELSE
  CREATE TABLE __LsInKalk (
    ID int
  );

DECLARE @LsKo TABLE (
  ID int PRIMARY KEY,
  [Status] nchar(1),
  VsaID int
);

INSERT INTO @LsKo (ID, [Status], VsaID)
SELECT LsKo.ID, LsKo.[Status], LsKo.VsaID
FROM LsKo
WHERE LsKo.[Status] >= N'Q'
  AND LsKo.SentToSAP = 0
  AND LsKo.InternKalkFix = 1
  AND (LEFT(LsKo.Referenz, 7) != N'INTERN_' OR LsKo.Referenz IS NULL) /* Umlagerungs-LS ausnehmen, diese werden vom Modul SAPSENDSTOCKTRANSACTION übertragen */
;

INSERT INTO __LsInKalk (ID)
SELECT LsKo.ID
FROM @LsKo LsKo
JOIN Vsa ON LsKo.VsaID = Vsa.ID
JOIN Kunden ON Vsa.KundenID = Kunden.ID
JOIN KdGf ON Kunden.KdGFID = KdGf.ID
JOIN Firma ON Kunden.FirmaID = Firma.ID
WHERE (
    (Firma.SuchCode = N'FA14' AND KdGf.KurzBez IN (N'MED', N'GAST', N'JOB', N'SAEU', N'BM', N'RT', N'MIC'))
    OR
    (Firma.SuchCode IN (N'SMP', N'SMKR', N'SMSK', N'SMRO', N'BUDA', N'SMRS', N'SMSL',N'SMHR', N'SMPL'))
  );

GO

### **Waschlohn übertragen**

Modul-Aufruf im AdvanTex: INKALKSEND;20210101;20210430;\_\_LsInKalk  
Datumswerte entsprechend anpassen!